In [1]:
import os
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import AveragePooling2D
from keras.utils import np_utils
from keras.applications.xception import Xception
from keras.optimizers import SGD
from keras import callbacks
from keras.backend import tensorflow_backend as backend
import keras.backend as K
import json

# 画像サイズ．ResNetを使う時は224
img_size = 224
batch_size = 32
#以下ディレクトリに入っている画像を読み込む
root_dir = "./face/"
#学習データを何周するか
epochs=50
#ログファイル
log_filepath="./logs/"
#学習したモデル
ModelWeightData="./face/face-model.h5"
ModelArcData="./face/face.json"
classFile="./face/categories.json"
#学習率(SGD(lr=???))
learning_rate=0.01
#活性化関数
activation='softmax'
#重みの初期化
kernel_init='glorot_uniform'


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
    #学習画像データを水増し（データ拡張）を行う
    mizumashi_data=ImageDataGenerator(rotation_range=180,width_shift_range=0.2)
    mizumashi_generator=mizumashi_data.flow_from_directory(directory=root_dir,target_size=(img_size,img_size),batch_size=batch_size,shuffle=True)
    #テスト画像データを水増しする。
    val_datagen=ImageDataGenerator(rotation_range=180,width_shift_range=0.2)
    val_gen=val_generator=val_datagen.flow_from_directory(directory=root_dir,target_size=(img_size,img_size),batch_size=batch_size,shuffle=False)
    valX,valy=val_gen.next()


Found 1135 images belonging to 12 classes.
Found 1135 images belonging to 12 classes.


In [3]:
    base_model = Xception(weights='imagenet', include_top=False,
                         input_tensor=Input(shape=(img_size,img_size, 3)))
   #base_model.summary()
    x=base_model.output
    #入力を平滑化
    x=Flatten()(x)


In [4]:
    predictions = Dense(mizumashi_generator.num_classes,kernel_initializer=kernel_init, activation=activation)(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    opt = SGD(lr=learning_rate)

In [5]:
#評価指標を定義（自分のプログラムで使用する際はこの部分をコピペすれば良い。呼び出し方法はmodel.compile(...,metrics[の部分参照])
def Precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def Recall(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def F1(y_true, y_pred):
    def Precision(y_true, y_pred):
  
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def Recall(y_true, y_pred):
       
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [6]:
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy',F1,Recall,Precision])
    tb_cb=keras.callbacks.TensorBoard(log_dir=log_filepath,histogram_freq=0)
    es_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=0,verbose=1,mode='auto')
    cbks=[tb_cb,es_cb]
    history = model.fit_generator(mizumashi_generator,
                                  validation_data=val_generator,
                                  steps_per_epoch=mizumashi_generator.samples// batch_size,
                                  validation_steps=val_generator.samples // batch_size,
                                  epochs=epochs,callbacks=cbks,
                                  verbose=1)
    score = model.evaluate(valX, valy,batch_size=batch_size)
    print('loss=', score[0])
    print('accuracy=', score[1])
    print('F1=', score[2])
    print('Recall',score[3])
    print('Precision',score[4])

Epoch 1/50
35/35 [==============================] - 29s 824ms/step - loss: 1.5303 - acc: 0.5704 - F1: nan - Recall: 0.3747 - Precision: 0.7271 - val_loss: 0.5043 - val_acc: 0.8607 - val_F1: 0.8298 - val_Recall: 0.7598 - val_Precision: 0.9279
Epoch 2/50
35/35 [==============================] - 27s 779ms/step - loss: 0.2664 - acc: 0.9428 - F1: 0.9276 - Recall: 0.8847 - Precision: 0.9783 - val_loss: 0.1366 - val_acc: 0.9755 - val_F1: 0.9710 - val_Recall: 0.9529 - val_Precision: 0.9911
Epoch 3/50
35/35 [==============================] - 27s 785ms/step - loss: 0.1205 - acc: 0.9705 - F1: 0.9681 - Recall: 0.9544 - Precision: 0.9825 - val_loss: 0.0595 - val_acc: 0.9909 - val_F1: 0.9871 - val_Recall: 0.9801 - val_Precision: 0.9945
Epoch 4/50
35/35 [==============================] - 28s 788ms/step - loss: 0.0583 - acc: 0.9937 - F1: 0.9900 - Recall: 0.9866 - Precision: 0.9936 - val_loss: 0.0457 - val_acc: 0.9955 - val_F1: 0.9911 - val_Recall: 0.9828 - val_Precision: 1.0000
Epoch 5/50
35/35 [=====

In [7]:
%%bash
echo "OK"|/home/kagamiwomiru/notiSlack.sh